En ayant à présent assez d’explorer sans but précis, vous vous fixez un petit challenge ! Vous prenez les données des arbres remarquables de Paris et répondez à plusieurs questions que vous vous posez :

    Quel âge a notre plus vieil ami?
    Quel est le volume du plus grand arbre ?
    Quelle est la hauteur moyenne de ces spécimens ?
    Dans quel arrondissement y a-t-il le plus d’arbres ?
    Combien y a-t-il d'arbres au Cimetière du Père Lachaise ?



In [129]:
from pyspark import SparkConf, SparkContext
from datetime import datetime
# Créez une configuration Spark:
conf = SparkConf().setAppName("SimpleApp").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [130]:
# Chargement des données CSV dans un RDD
rdd_arbres = sc.textFile("arbresremarquablesparis.csv")

In [162]:
rdd_arbres.take(1)

["Geo point;idbase;domanialite;arrondissement;complement adresse;numero;adresse;circonference en cm;hauteur en m;stade développement;pépinière;genre;espèce;varieteoucultivar;date de plantation;libellé Français;ID Base;ID arbre;Site;Adresse;Complément d'adresse;Arrondissement;Domanialité;Dénomination usuelle;Dénomination botanique;Autorité taxonomique;Année de plantation;Qualification remarquable;Résumé;Descriptif;Numéro de délibération;Date de la délibération;Label national;Panonceau;Photo 1;Copyright 1"]

In [163]:
# En-tête du fichier CSV : Nous extrayons la première ligne (l'en-tête) du RDD. (les colonnes)
header = rdd_arbres.first()
header

"Geo point;idbase;domanialite;arrondissement;complement adresse;numero;adresse;circonference en cm;hauteur en m;stade développement;pépinière;genre;espèce;varieteoucultivar;date de plantation;libellé Français;ID Base;ID arbre;Site;Adresse;Complément d'adresse;Arrondissement;Domanialité;Dénomination usuelle;Dénomination botanique;Autorité taxonomique;Année de plantation;Qualification remarquable;Résumé;Descriptif;Numéro de délibération;Date de la délibération;Label national;Panonceau;Photo 1;Copyright 1"

In [ ]:
separateur= ';'

In [153]:
[(n, i) for i, n in enumerate(header.split(separateur))]

[('Geo point', 0),
 ('idbase', 1),
 ('domanialite', 2),
 ('arrondissement', 3),
 ('complement adresse', 4),
 ('numero', 5),
 ('adresse', 6),
 ('circonference en cm', 7),
 ('hauteur en m', 8),
 ('stade développement', 9),
 ('pépinière', 10),
 ('genre', 11),
 ('espèce', 12),
 ('varieteoucultivar', 13),
 ('date de plantation', 14),
 ('libellé Français', 15),
 ('ID Base', 16),
 ('ID arbre', 17),
 ('Site', 18),
 ('Adresse', 19),
 ("Complément d'adresse", 20),
 ('Arrondissement', 21),
 ('Domanialité', 22),
 ('Dénomination usuelle', 23),
 ('Dénomination botanique', 24),
 ('Autorité taxonomique', 25),
 ('Année de plantation', 26),
 ('Qualification remarquable', 27),
 ('Résumé', 28),
 ('Descriptif', 29),
 ('Numéro de délibération', 30),
 ('Date de la délibération', 31),
 ('Label national', 32),
 ('Panonceau', 33),
 ('Photo 1', 34),
 ('Copyright 1', 35)]

In [155]:
df_arbres = rdd_arbres.filter(lambda line: line != header).map(lambda line: line.split(separateur))

## Quel âge a notre plus vieil ami?

In [156]:
def calculer_age(x):
    return datetime.now().year - int (x[:4])

In [157]:
(
    rdd_arbres
    .filter(lambda line: line != header)
    .map(lambda line: line.split(separateur))
    .map(lambda line: (line[3], line[6], float(line[7]), float(line[8]), calculer_age(line[14]), str(line[18]))) # garder 3, 6, 7, 8, 14 et 18
    .sortBy(lambda line: line[4], ascending= False) # trouver le plus vieux
).take(1)

[('PARIS 5E ARRDT',
  'SQUARE RENE VIVIANI MONTEBELLO / 2 RUE DU FOUARRE',
  365.0,
  15.0,
  422,
  'Square René Viviani')]

## Quel est le volume du plus grand arbre ?

In [158]:
(
    rdd_arbres
    .filter(lambda line: line != header)
    .map(lambda line: line.split(separateur))
    .map(lambda line: (line[3], line[6], float(line[7]), float(line[8]), calculer_age(line[14]), str(line[18]))) # garder 3, 6, 7, 8, 14 et 18
    .sortBy(lambda line: line[2], ascending = False) # trouver la plus volumeuse
).take(1)

[('PARIS 19E ARRDT',
  'PARC DES BUTTES CHAUMONT',
  740.0,
  30.0,
  324,
  'Parc des Buttes Chaumont')]

## La hauteur moyenne de ces spécimens:

In [159]:
(
    rdd_arbres
    .filter(lambda line: line != header)
    .map(lambda line: line.split(separateur))
    .map(lambda line: (line[3], line[6], float(line[7]), float(line[8]), calculer_age(line[14]))) # garder 3, 6, 7, 8, 14 et 18
    .map(lambda line : line[3]).mean() # trouver la moyenne
)

20.129032258064512

## Dans quel arrondissement y a-t-il le plus d’arbres ?

In [160]:
(
    rdd_arbres
    .filter(lambda line: line != header)
    .map(lambda line: line.split(separateur))
    .map(lambda line: (line[3], line[6], float(line[7]), float(line[8]), calculer_age(line[14]), str(line[18]))) # garder 3, 6, 7, 8, 14 et 18
    .map(lambda line: (line[0], 1))
    .reduceByKey(lambda a, b: a + b)
    .max(key=lambda x: x[1]) #Trier les résultats et obtenir l'arrondissement avec le plus grand nombre d'arbres :
)

('BOIS DE VINCENNES', 33)

## Combien y a-t-il d'arbres au Cimetière du Père Lachaise ?


In [161]:
(
    rdd_arbres
    .filter(lambda line: line != header)
    .map(lambda line: line.split(separateur))
    .map(lambda line: (line[3], line[6], float(line[7]), float(line[8]), calculer_age(line[14]), str(line[18]))) # garder 3, 6, 7, 8, 14 et 18
    .filter(lambda line: 'Cimetière du Père Lachaise' in line[5])
    .count()  ## Filtrer les arbres situés dans le "Cimetière du Père Lachaise" et compter le nombre d'arbres
)

9

Arrêt du contexte Spark

In [126]:
sc.stop()